<a href="https://colab.research.google.com/github/LayanAlrashoud/ClassifyAndSummary/blob/main/summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
folder_path = '/content/drive/MyDrive/Data'

In [ ]:
!pip install datasets

In [ ]:
import os
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments

In [ ]:
# 1. دالة تحميل البيانات من المجلدات وتنظيمها
def load_texts_for_fine_tuning(data_folder):
    inputs = []
    targets = []

    categories = ['articlesEconomy', 'articlesLocal', 'articlesInternational', 'articlesSports', 'articlesReligion', 'articlesCulture']

    for category in categories:
        folder_path = os.path.join(data_folder, category)

        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)

            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()

                # المدخل هو النص الأصلي
                input_text = text

                # الهدف هو أول 200 حرف من النص الأصلي
                target_text = text[:200]

                inputs.append(input_text)
                targets.append(target_text)

    # تحويل البيانات إلى قاموس
    data = {'input': inputs, 'target': targets}
    return data

# 3. تحميل البيانات من المجلد
data_folder ='/content/drive/MyDrive/Data'
data = load_texts_for_fine_tuning(data_folder)

# 4. تحويل البيانات إلى Dataset من مكتبة datasets
dataset = Dataset.from_dict(data)

In [ ]:
# 5. تقسيم البيانات إلى تدريب واختبار
split_dataset = dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

# 6. تحميل النموذج والـ tokenizer
model_name = "eslamxm/AraBART-finetuned-ar-wikilingua"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# 7. دالة تحضير البيانات للتدريب
def preprocess_function(examples):
    inputs = examples['input']
    targets = examples['target']

    # ترميز المدخلات
    model_inputs = tokenizer(inputs, max_length=512, padding="max_length", truncation=True)

    # ترميز النصوص المستهدفة
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, padding="max_length", truncation=True)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs


# 8. تطبيق عملية التحضير على بيانات التدريب والاختبار
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# 9. إعدادات التدريب باستخدام Seq2SeqTrainer
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",  # مسار حفظ النموذج المدرب
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,  # يمكنك زيادة عدد epochs حسب الحاجة
    predict_with_generate=True
)

# 10. إعداد المدرب (Trainer)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
)

# 11. بدء التدريب
trainer.train()

# 12. تقييم النموذج (اختياري)
trainer.evaluate()

# 13. حفظ النموذج المدرب
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")


tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.037600,0.034615
2,0.038300,0.027531
3,0.031900,0.025188


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generatio

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/sentencepiece.bpe.model',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [ ]:
!pip install rouge-score nltk

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=0d5d7c3ced933f8dffd01fe22158c0a50910e2420dd9e6a69c7953d1a3f0957c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import torch

# تحميل النموذج المدرب بعد Fine-tuning
model_name = "./fine_tuned_model"  # مسار النموذج المدرب
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to('cuda')

# 1. إعداد دالة لحساب BLEU score
def compute_bleu(reference, generated):
    smooth = SmoothingFunction().method4
    reference_tokens = reference.split()
    generated_tokens = generated.split()
    return sentence_bleu([reference_tokens], generated_tokens, smoothing_function=smooth)

# 2. إعداد دالة لحساب ROUGE score
def compute_rouge(reference, generated):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, generated)
    return scores

# 3. توليد التلخيص باستخدام النموذج المدرب
def generate_summary(input_text):
    # ترميز النص المدخل
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512).to('cuda')

    outputs = model.generate(
        inputs['input_ids'],
        max_length=200,  # تحديد الحد الأقصى لطول التلخيص
        min_length=50,  # الحد الأدنى لطول التلخيص
        num_beams=10,  # تحسين الجودة باستخدام الشعاع
        no_repeat_ngram_size=2,  # منع تكرار نفس الجمل
        early_stopping=True
    )

    # فك ترميز النص المولد (التلخيص)
    generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_summary

input_text = """
مع تطور التكنولوجيا الحديثة، أصبحت الهواتف الذكية جزءًا لا يتجزأ من حياتنا اليومية. فهي تتيح لنا التواصل مع الآخرين، الوصول إلى الإنترنت، التقاط الصور، تسجيل الفيديوهات، والعديد من الميزات الأخرى. ومع ذلك، فإن هذه التكنولوجيا تأتي مع تحديات كبيرة مثل مشكلات الخصوصية وإدمان استخدام الأجهزة. يمكن أن تؤدي الهواتف الذكية إلى فقدان الوقت والتركيز، كما أنها قد تؤثر على العلاقات الشخصية إذا أسيء استخدامها. بالرغم من هذه التحديات، تظل الهواتف الذكية أداة مهمة لتسهيل الحياة اليومية.
"""

reference_text = "الهواتف الذكية أصبحت جزءًا من حياتنا اليومية، حيث توفر العديد من المزايا مثل التواصل والتقاط الصور، لكنها تطرح تحديات مثل إدمان الأجهزة ومشكلات الخصوصية."


# توليد التلخيص
generated_summary = generate_summary(input_text)

# 5. حساب BLEU score
bleu_score = compute_bleu(reference_text, generated_summary)
print(f"BLEU score: {bleu_score}")

# 6. حساب ROUGE score
rouge_scores = compute_rouge(reference_text, generated_summary)
print(f"ROUGE scores: {rouge_scores}")

print("\nالنص المولد:", generated_summary)


BLEU score: 0.16859249344043362
ROUGE scores: {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0, recall=0, fmeasure=0)}

النص المولد: مع تطور التكنولوجيا الحديثة، أصبحت الهواتف الذكية جزءا لا يتجزأ من حياتنا اليومية فهي تتيح لنا التواصل مع الآخرين، الوصول إلى الإنترنت، التقاط الصور، تسجيل الفيديوهات، والعديد من الميزات الأخرى 


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import torch

# 1. إعداد دالة لحساب BLEU score
def compute_bleu(reference, generated):
    smooth = SmoothingFunction().method4
    reference_tokens = reference.split()
    generated_tokens = generated.split()
    return sentence_bleu([reference_tokens], generated_tokens, smoothing_function=smooth)

# 2. إعداد دالة لحساب ROUGE score
def compute_rouge(reference, generated):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, generated)
    return scores

# 3. اختبار النموذج على نصوص جديدة (توليد تلخيصات)
def generate_summary(input_text):
    # ترميز النص المدخل
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512).to('cuda')

    outputs = model.generate(
    inputs['input_ids'],
    max_length=100,  # تحديد الحد الأقصى لطول التلخيص ليكون أقصر
    min_length=50,  # الحد الأدنى لطول التلخيص
    num_beams=5,  # تحسين الجودة باستخدام الشعاع
    no_repeat_ngram_size=2,  # منع تكرار نفس الجمل
    early_stopping=True
)


    # فك ترميز النص المولد (التلخيص)
    generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_summary

# 4. إعداد نصوص المدخلات والنصوص المرجعية
input_text = """
التعليم عبر الإنترنت أصبح اليوم من أهم طرق التعليم الحديثة التي توفر مرونة للطلاب في مختلف أنحاء العالم.
فمن خلال المنصات التعليمية الإلكترونية، يستطيع الطلاب الحصول على المحتوى التعليمي في أي وقت ومن أي مكان.
"""

# النص المرجعي
reference_text = "التعليم عبر الإنترنت أصبح اليوم من أهم طرق التعليم الحديثة التي توفر مرونة للطلاب في مختلف أنحاء العالم فمن خلال المنصات التعليمية الإلكترونية، يستطيع الطلاب الحصول علي المحتوى التعليمي في اي وقت"

# توليد التلخيص
generated_summary = generate_summary(input_text)

# 5. حساب BLEU score
bleu_score = compute_bleu(reference_text, generated_summary)
print(f"BLEU score: {bleu_score}")

# 6. حساب ROUGE score
rouge_scores = compute_rouge(reference_text, generated_summary)
print(f"ROUGE scores: {rouge_scores}")

print("\nالنص المولد:", generated_summary)


BLEU score: 0.7528670579965006
ROUGE scores: {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0, recall=0, fmeasure=0)}

النص المولد: التعليم عبر الإنترنت أصبح اليوم من أهم طرق التعليم الحديثة التي توفر مرونة للطلاب في مختلف أنحاء العالم فمن خلال المنصات التعليمية الإلكترونية، يستطيع الطلاب الحصول علي المحتوى التعليمي في اي وقت ومن أي مكان ومن اي مكان يستحوذ علي شهاده جامعيه


In [ ]:
# النص المدخل
input_text = """
التعليم عبر الإنترنت أصبح اليوم من أهم طرق التعليم الحديثة التي توفر مرونة للطلاب في مختلف أنحاء العالم.
هذه الطريقة تقدم فوائد عديدة مثل توفير الوقت والجهد، وتمكين الطلاب من متابعة الدروس بالسرعة التي تناسبهم.
ومع ذلك، فإن هذا النظام يواجه تحديات عديدة مثل ضرورة توفر اتصال بالإنترنت عالي السرعة وصعوبة التفاعل المباشر
مع المعلمين والزملاء. لكن بشكل عام، فإن التعليم عبر الإنترنت يمثل خطوة هامة نحو مستقبل التعليم العصري.
"""

# ترميز النص المدخل
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)

# نقل المدخلات إلى الـ GPU
inputs = {key: value.to('cuda') for key, value in inputs.items()}

# نقل النموذج إلى الـ GPU
model = model.to('cuda')

outputs = model.generate(
    inputs['input_ids'],
    max_length=500,  # جرب زيادة هذا إلى 500
    num_beams=10,  # ابق على استخدام الشعاع لتحسين الجودة
    no_repeat_ngram_size=2,  # منع تكرار نفس الجمل
    early_stopping=True  # إيقاف التوليد إذا انتهى
)


# فك ترميز النص المولد (التلخيص)
generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("التلخيص:", generated_summary)


التلخيص: التعليم عبر الإنترنت أصبح اليوم من أهم طرق التعليم الحديثة التي توفر مرونة للطلاب في مختلف أنحاء العالم. هذه الطريقة تقدم فوائد عديدة مثل توفير الوقت والجهد، وتمكين الطلاب من متابعة الدروس بالسرعة
